In [1]:
import numpy as np

In [2]:
MOVES = {'U': np.array([0, -1]), 
         'D': np.array([0, 1]),
         'R': np.array([1, 0]),
         'L': np.array([-1, 0])}
DIRECTIONS = set(['U', 'D', 'R', 'L'])

In [55]:
# TEST CASE 1
_map = \
"""###################
###################
###################
###################
###............####
###################
###################
###################
###################
###################"""
robots = [(3, 4, 'R')]

# TEST CASE 2
_map = \
"""###################
###################
###............####
###################
###################
###################
###################
###............####
###################
###################"""
robots = [(3, 2, 'R'), (14, 7, 'L')]

# TEST CASE 3
_map = \
"""###################
###################
###################
###################
###............####
###################
###################
###################
###################
###################"""
robots = [(3, 4, 'L')]

# TEST CASE 4
_map = \
"""###################
###################
###################
###################
###............####
###################
###################
###################
###################
###################"""
robots = [(3, 4, 'L'), (14, 4, 'R')]

# TEST CASE 5
_map = \
"""###################
###################
###################
########...########
########...########
########...########
###################
###################
###################
###################"""
robots = [(9, 5, 'U')]

# TEST CASE 6
_map = \
"""###################
#.................#
#.###############.#
#.#.............#.#
#.#.###########.#.#
#.#.###########.#.#
#.#.............#.#
#.###############.#
#.................#
###################"""
robots = [(1, 1, 'R'), (15, 6, 'L')]

# TEST CASE 7
_map = \
"""###################
#.................#
#.###############.#
#.###############.#
#.###############.#
#.###############.#
#.###############.#
#D###############.#
#.................#
###################"""
robots = [(1, 1, 'R')]

# TEST CASE 8
_map = \
"""#########.#########
#########D#########
#########.#########
#########.#########
......R......R.....
#########.#########
#########.#########
#########D#########
#########.#########
#########.#########"""
robots = [(10, 4, 'R')]

# TEST CASE 9
_map = \
"""###################
#.......###.......#
#.#########.#######
#.#########.#######
#.#########.#######
#.#########.##....#
#.#########.#####.#
#.#########.#####.#
#.......###.......#
###################"""
robots = [(7, 1, 'L'), (17, 1, 'L'), (14, 5, 'R'), (7, 8, 'L')]

# TEST CASE 10
_map = \
"""###################
#...#...#...#...###
#...#...#...#...###
#...#...#...#...###
###################
#...#...#...#...###
#...#...#...#...###
#...#...#...#...###
###################
###################"""
robots = [(1, 1, 'R'), (6, 1, 'D'), (11, 1, 'L'), (14, 2, 'D'), (14, 6, 'U'),
          (1, 7, 'R'), (6, 7, 'U'), (11, 7, 'L')]

# TEST CASE 11
_map = \
"""L..#############..U
...#############...
...#############...
###################
###################
###################
###################
...#############...
...#############...
D..#############..R"""
robots = [(2, 2, 'L'), (16, 2, 'U'), (2, 7, 'D'), (16, 7, 'R')]

# TEST CASE 12
_map = \
"""..................#
#################.#
................#.#
.##############.#.#
.#............#.#.#
.#.############.#.#
.#..............#.#
.################.#
..................#
###################"""
robots = [(0, 0, 'R')]

# TEST CASE 13
_map = \
"""..................#
#################.#
................#.#
.##############.#.#
.#............#.#.#
.#.############.#.#
.#..............#.#
.################.#
..................#
###################"""
robots = [(0, 0, 'R'), (0, 2, 'R'), (2, 4, 'R'), (2, 6, 'R'), (0, 8, 'R')]

# TEST CASE 14
_map = \
"""###################
#.RRRRRRRULLLLLLL.#
#DRDRDRDRDLDLDLDLD#
#DUDUDUDUDUDUDUDUD#
#RURURURU.ULULULUL#
#RDRDRDRD.DLDLDLDL#
#UDUDUDUDUDUDUDUDU#
#URURURURULULULULU#
#.RRRRRRRDLLLLLLL.#
###################"""
robots = [(1, 1, 'L'), (17, 1, 'R'), (1, 8, 'L'), (17, 8, 'R')]

# TEST CASE 15
_map = \
"""#.................#
#.#.#.#.#.#.#.#.#.#
#.................#
#.#.#.#.#.#.#.#.#.#
#.................#
#.#.#.#.#.#.#.#.#.#
#.................#
#.#.#.#.#.#.#.#.#.#
#.................#
###################"""
robots = [(1, 0, 'R'), (17, 0, 'D'), (1, 8, 'U'), (17, 8, 'L')]

# TEST CASE 16
_map = \
"""...................
#.#.#.#.#.#.#.#.#.#
...................
#.#.#.#.#.#.#.#.#.#
...................
#.#.#.#.#.#.#.#.#.#
...................
#.#.#.#.#.#.#.#.#.#
...................
#.#.#.#.#.#.#.#.#.#"""
robots = [(1, 0, 'R'), (17, 0, 'D'), (1, 8, 'U'), (17, 8, 'L')]

# TEST CASE 17
_map = \
"""###################
#.................#
#.................#
#.................#
RRRRRRRRRRRRRRRRRRR
LLLLLLLLLLLLLLLLLLL
#.................#
#.................#
#.................#
###################"""
robots = [(1, 1, 'R'), (9, 1, 'D'), (17, 2, 'L'), (1, 3, 'R'), (17, 6, 'L'), (1, 7, 'R'), (9, 8, 'U'), (17, 8, 'L')]

# TEST CASE 18
_map = \
"""#.....#######.###.#
#.###########.###.#
#.#...#######.....#
#.###.#######.###.#
#.....#######.###.#
#.###########.....#
#.###########.#####
#.###########...###
#.###########.#####
#.....#######.#####"""
robots = [(5, 0, 'L'), (13, 0, 'D'), (17, 0, 'D'), (3, 2, 'R'), (13, 4, 'U'), (17, 4, 'U'), (1, 5, 'D'), (17, 5, 'L'), (5, 9, 'L'), (13, 9, 'U')]

# TEST CASE 19
_map = \
"""##.D.....U.....D.##
#..D.....#.....D..#
...D..LL###RR..D...
...U.....#.....U...
...U.....D.....U...
...U.....D.....U...
...#.....D.....#...
LL###RR..D..LL###RR
#..#.....U.....#..#
##.D.....U.....D.##"""
robots = [(1, 1, 'D'), (8, 1, 'U'), (10, 1, 'U'), (17, 1, 'D'), (8, 3, 'D'), (10, 3, 'D'), (7, 7, 'D'), (11, 7, 'D'), (1, 8, 'R'), (17, 8, 'L')]

# TEST CASE 20
_map = \
"""...................
..............R....
..............RR...
..............RRR..
RRRRRRRRRRRRRRRRRR.
RRRRRRRRRRRRRRRRRRR
RRRRRRRRRRRRRRRRRR.
..............RRR..
..............RR...
..............R...."""
robots = [(0, 2, 'R'), (0, 3, 'R'), (0, 7, 'R'), (0, 8, 'R')]

# TEST CASE 21
_map = \
"""...................
...................
...................
U#DDDDDDD#DDDDDDD#U
UU#DDDDD#U#DDDDD#UU
UUU#DDD#UUU#DDD#UUU
UUUU#D#UUUUU#D#UUUU
UUUUU#UUUUUUU#UUUUU
...................
..................."""
robots = [(1, 2, 'U'), (9, 2, 'U'), (17, 2, 'U'), (5, 8, 'D'), (13, 8, 'D')]

# TEST CASE 22
_map = \
"""RRRRRRRRR#RRRRRRRRD
U........U........D
U........U........D
U........U........D
#LLL..............D
U..............RRR#
U........D........D
U........D........D
U........D........D
ULLLLLLLL#LLLLLLLLL"""
robots = [(1, 1, 'R'), (17, 1, 'D'), (4, 4, 'L'), (14, 5, 'R'), (1, 8, 'U'), (17, 8, 'L')]

# TEST CASE 23
_map = \
"""###################
#........#........#
#.................#
#........#........#
####.#########.####
####.#########.####
#........#........#
#.................#
#........#........#
###################"""
robots = [(1, 1, 'R'), (17, 1, 'D'), (1, 8, 'U'), (17, 8, 'L')]

# TEST CASE 24
_map = \
"""..#######.#######..
.########.########.
.########.########.
.#######...#######.
...................
.###.###...###.###.
.##...###.###...##.
......###.###......
.##...###.###...##.
..#######.#######.."""
robots = [(8, 4, 'L'), (10, 4, 'R'), (4, 7, 'L'), (14, 7, 'R')]

# TEST CASE 25
_map = \
"""...................
.#######...#######.
.#.........#.......
.#.........#.......
.#######...#.......
.......#...#.......
.......#...#.......
.#######...#######.
...................
RRRRRRRRRRRRRRRRRRR"""
robots = [(2, 2, 'R'), (12, 2, 'D'), (2, 3, 'R'), (6, 5, 'L'), (6, 6, 'L'), (12, 6, 'U')]

# TEST CASE 26
_map = \
"""#....R........D....
#.....R......D.....
#......R....D......
#.......R..D.......
#........RD........
#........UL........
#.......U..L.......
#......U....L......
#.....U......L.....
#....U........L...."""
robots = [(1, 0, 'R'), (18, 0, 'L'), (1, 9, 'R'), (18, 9, 'L')]

# TEST CASE 27
_map = \
"""###################
.........#.........
#U.......#.......U#
.........#.........
#U.......#.......U#
#D.......#.......D#
.........#.........
#D.......#.......D#
.........#.........
###################"""
robots = [(8, 1, 'L'), (10, 1, 'R'), (8, 8, 'L'), (10, 8, 'R')]

# TEST CASE 28
_map = \
"""#...#...#...#...#..
#.#.#.#.#.#.#.#.#.#
#.#.#.#.#.#.#.#.#.#
#.#.#.#.#.#.#.#.#.#
#.#.#.#.#.#.#.#.#.#
#.#.#.#.#.#.#.#.#.#
#.#.#.#.#.#.#.#.#.#
#.#.#.#.#.#.#.#.#.#
#.#.#.#.#.#.#.#.#.#
..#...#...#...#...#"""
robots = [(18, 0, 'L'), (9, 4, 'U'), (9, 5, 'D'), (0, 9, 'R')]

# TEST CASE 29
_map = \
"""########UUU########
####...........####
#.................#
#.................#
L.................R
L.................R
#.................#
#.................#
####...........####
########DDD########"""
robots = [(1, 2, 'L'), (17, 2, 'R'), (1, 7, 'L'), (17, 7, 'R')]

# TEST CASE 30
_map = \
"""#........L........#
#........R........#
#........L........#
#........R........#
#DUDUDUDU#UDUDUDUD#
#........R........#
#........L........#
#........R........#
#........L........#
###################"""
robots = [(1, 0, 'R'), (17, 0, 'D'), (1, 8, 'U'), (17, 8, 'L')]

In [56]:
_map = np.array([list(l) for l in _map.split('\n')])

In [57]:
def update_direction(x, y, d):
    return (d, _map[y, x])[_map[y, x] in DIRECTIONS]

def roll_coordinates(x, y):
    x = (x, 18)[x < 0]
    y = (y, 9)[y < 0]
    x = (x, 0)[x > 18]
    y = (y, 0)[y > 9]
    return x, y

In [58]:
score = 0
for robot in robots:
    visited_states = set()
    x, y, d = robot
    d = update_direction(x, y, d)
    is_live = True
    robot_score = 0
    while is_live:
        robot_score += 1
        visited_states.add((x, y, d))
        x += MOVES[d][0]
        y += MOVES[d][1]
        x, y = roll_coordinates(x, y)
        d = update_direction(x, y, d)
        is_live = _map[y, x] != '#' and (x, y, d) not in visited_states
    score += robot_score
    
score

/home/andrey/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  """
/home/andrey/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  
/home/andrey/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  import sys
/home/andrey/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  


52